<a href="https://colab.research.google.com/github/Vanashreeghegde/Medical-AI-VIT-/blob/main/MedicalAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers datasets torch torchvision evaluate gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.8 MB/s eta 0:00:00


In [2]:
import torch
import numpy as np
import evaluate
import gradio as gr
from datasets import load_dataset
from transformers import (
    ViTImageProcessor,
    ViTForImageClassification,
    TrainingArguments,
    Trainer
)

In [3]:
import torch.nn.functional as F

In [4]:
raw_dataset = load_dataset("zacharielegault/PatchCamelyon", split='train', streaming=True)
dataset_subset = list(raw_dataset.take(5000))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

In [5]:
model_name = "google/vit-base-patch16-224"
processor = ViTImageProcessor.from_pretrained(model_name)

def transform_medical_images(examples):
    inputs = processor(examples['image'], return_tensors='pt')
    inputs['labels'] = examples['label']
    # .squeeze() here prevents the dimension error later!
    inputs['pixel_values'] = inputs['pixel_values'].squeeze()
    return inputs

processed_dataset = [transform_medical_images(ex) for ex in dataset_subset]

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

In [6]:
id2label = {0: "Healthy", 1: "Malignant"}
label2id = {"Healthy": 0, "Malignant": 1}

model = ViTForImageClassification.from_pretrained(
    model_name,
    num_labels=2,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)



config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_features=3072, bias=True)
            (intermed

In [7]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(
    output_dir="./vit-cancer-model",
    per_device_train_batch_size=16,
    num_train_epochs=3,
    learning_rate=5e-6,
    weight_decay=0.01,
    logging_steps=10,
    eval_strategy="no",
    save_strategy="epoch",
    fp16=True if torch.cuda.is_available() else False,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset,
    compute_metrics=compute_metrics,
)

In [8]:
print("Starting Training...")
trainer.train()
print("Training Complete!")

Starting Training...


Step,Training Loss
10,0.573700
20,0.162400
30,0.035700
40,0.011600
50,0.006500
60,0.002800
70,0.002400
80,0.002200
90,0.002200
100,0.001400


Step,Training Loss
10,0.573700
20,0.162400
30,0.035700
40,0.011600
50,0.006500
60,0.002800
70,0.002400
80,0.002200
90,0.002200
100,0.001400


Training Complete!


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
model.push_to_hub("cancer-ai-model")
processor.push_to_hub("cancer-ai-model")